# Main Commands

## Fetch Images

### Download card images from Scryfall queries

In [ ]:
from art_scan import process_query

queries = [

]

force_scryfall = [

]

force_mtgpics = [

]

[process_query(q) for q in queries]
[process_query(q, skip_mtgpics=True) for q in force_scryfall]
[process_query(q, skip_scryfall=True) for q in force_mtgpics]

## Fetch Set Symbols

### Parse Keyrune website cheatsheet and print out UTF8 representation

In [ ]:
from bs4 import BeautifulSoup
import requests

url = "https://keyrune.andrewgioia.com/cheatsheet.html"

response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
set_symbol_spans = soup.findAll("span", {"class": "utf"})
print(f"Found {len(set_symbol_spans)} icons.")

In [ ]:
set_symbols = {}
for set_symbol_span in set_symbol_spans:
    set_symbol = str(set_symbol_span.findChild("i").get_text())
    try:
        set_code = set_symbol_span.get_text().split("ss-")[1].split(" ")[0].upper()
    except IndexError as e:
        continue
    finally:
        set_symbols[set_code] = set_symbol

for set_code, set_symbol in set_symbols.items():
    print(f"\"{set_code}\": \"{set_symbol}\",")

## Inventory Management

### Check for image files

In [ ]:
from lib.helpers import os_helpers

ART_FOLDER = "C:/Users/evanh/My Drive/MTG/Proxies/art"

to_find = [
    "Crucible of Worlds"
]

os_helpers.find_missing_files(ART_FOLDER, to_find)

### Generate inventory as TSV

In [ ]:
import operator
from lib.classes import InventoryCard
from lib.helpers import os_helpers, scryfall_helpers

PROXY_FOLDER = "C:/Users/evanh/My Drive/MTG/Proxies/mpc/all"
IGNORED_FILES = ["desktop.ini", "inventory.tsv", "done"]
INVENTORY_FILE = "C:/Users/evanh/My Drive/MTG/Proxies/mpc/inventory.tsv"

inventory_cards = []
for file_name in os_helpers.list_files(PROXY_FOLDER, ignore=IGNORED_FILES):
    print(file_name)
    name = file_name[:file_name.index("(")-1]
    artist_set = file_name[file_name.index("(")+1:file_name.index(")")]
    artist, set_code = artist_set.split(", ")
    
    try:
        frame = file_name[file_name.index("[")+1:file_name.index("]")]
    except ValueError:
        frame = "Normal"
    
    scryfall_card = scryfall_helpers.get_named_card(name, set_code)
    print(scryfall_card)
    
    inventory_card = InventoryCard(
        name=name,
        set_code=set_code,
        artist=artist,
        frame=frame,
        type_=scryfall_card.type_alt,
        modified=os_helpers.get_modified_date_utc(f"{PROXY_FOLDER}/{file_name}"),          
    )
    inventory_cards.append(inventory_card)

# Sort inventory to match destination spreadsheet
sorted_cards = sorted(inventory_cards, key=operator.itemgetter("type_", "name"))

# Write list of cards to TSV
with open(INVENTORY_FILE, "w") as f:
    for card in sorted_cards:
        f.write(card.tsv + "\n")
    print(f"Created inventory at: \"{INVENTORY_FILE}\"")

### Load decklist into inventory

In [ ]:
import json
from lib.classes import OrderCard

orders = {
    # "Evan": "orders/evan.json",
    # "Blake": "orders/blake.json",
    "Matt": "orders/matt.json",
    # "Scott": "orders/scott.json",
}

for name, list_file in orders.items():
    with open(list_file, "r") as f:
        order_list = json.load(f)
        [inventory.add_to_order(card) for card in OrderCard.import_list(order_list, name)]

## Order Generation

### Pull proxy list from Google Sheets (public)

In [ ]:
from lib import common
from lib.classes import Inventory

sheet_id = '1JbJy3kM34XUOPud4qPaAhp2luU0Qgidq4CPQVm0TTw0'
url = f'https://docs.google.com/spreadsheet/ccc?key={sheet_id}&output=csv'

proxy_csv_data = common.load_csv(url.format(sheet_id))
inventory = Inventory.from_csv(proxy_csv_data)
print(inventory)

### Pull proxy list from Google Sheets (private)

In [ ]:
from googleapiclient import discovery
from lib.google_auth import get_creds

args = dict(
    spreadsheetId='13ofTLZFmPMkT09Np2US8KF_JmTfgqdeQ5aeY16L43s0',
    range='Inventory!A:L',
)

service = discovery.build('sheets', 'v4', credentials=get_creds())
result = service.spreadsheets().values().get(**args).execute()
proxy_csv_data = list(result.get('values', []))
inventory = Inventory.from_csv(proxy_csv_data)
print(inventory)

### Find missing proxy files

In [ ]:
# TODO: Fix this now that proxies are not in subfolders

from lib.helpers import os_helpers

proxy_folder = "C:/Users/evanh/My Drive/MTG/Proxies/mpc-ready"

card_types = list(set([p.type for p in proxies_to_print]))
proxies_by_type = {
    type_: [p for p in proxies_to_print if p.type == type_]
    for type_ in card_types
}

missing_files = []
for card_type in card_types:
    folder_name = f"{proxy_folder}/{card_type}"
    proxy_file_names = [p.name for p in proxies_by_type[card_type]]
    missing_files += os_helpers.find_missing_files(folder_name, proxy_file_names)

print("\n".join(missing_files))

### Write out files for order

In [ ]:
proxies = [p for p in proxies_to_print if not p.is_mdfc]
# proxies = [p for p in proxies_to_print if p.is_mdfc]
len(proxies)

In [ ]:
# TODO: Fix this now that proxies are not in subfolders

from lib.helpers import inventory_helpers, os_helpers

folder_size = 612
prefix_name = "proxies_{number:02d}"
landing_folder = "C:/Users/evanh/Temp/landing"
output_folder = "C:/Users/evanh/Temp/mpc"
proxy_folder = "C:/Users/evanh/My Drive/MTG/Proxies/mpc-ready"

# Write all files out to landing zone
missing, duplicates = inventory_helpers.create_unique_proxies(proxies, proxy_folder, landing_folder)
if len(missing):
    print("Missing:\n\t{}".format('\n\t'.join([p.name for p in missing])))
if len(duplicates):
    print("Duplicates:\n\t{}".format('\n\t'.join([p.name for p in duplicates])))

# Move files into subfolders of fixed size (`folder_size`)
output_folders = os_helpers.split_files(landing_folder, output_folder, folder_size, prefix_name)
output_folders

### Move proxy files

In [ ]:
import os
from lib.helpers import os_helpers

art_folder = "C:/Users/evanh/My Drive/MTG/Proxies/art"
proxy_folder = "C:/Users/evanh/My Drive/MTG/Proxies/mpc-ready"
destination_folder = "C:/Users/evanh/My Drive/MTG/Proxies/mpc"

ignored_files = [
    "desktop.ini",
    "Custom",
    "inventory.tsv",
]

# Iterate through art files to hold details in memory
art = {}
for file_name in os_helpers.list_files(art_folder, ignore=ignored_files):
    try:
        name = file_name[:file_name.index("(")-1]
        artist = file_name[file_name.index("(")+1:file_name.index(")")]
        set_code = file_name[file_name.index("[")+1:file_name.index("]")]
    except ValueError as e:
        print(f"Unable to parse {file_name}")

    # Store results in a list to handle cards that have multiple versions
    if name not in art:
        art[name] = [(artist, set_code)]
    else:
        art[name].append((artist, set_code))

# Iterate through proxy files to rename & move them based on art files
for typed_proxy_folder in os_helpers.list_files(proxy_folder, ignore=ignored_files, include_base=True):
    # for original_proxy_file in os_helpers.list_files(typed_proxy_folder, ignore=ignored_files)[:3]:
    for original_proxy_file in os_helpers.list_files(typed_proxy_folder, ignored_files):
        # Extract attributes from proxy (if available)
        file_extension = original_proxy_file.split(".")[-1]
        proxy_file = "".join(".".join(original_proxy_file.split(".")[:-1]))  # Trim file extension

        try:
            proxy_artist = proxy_file[proxy_file.index("[")+1:proxy_file.index("]")]
            proxy_file = proxy_file.replace(f" [{proxy_artist}]", "")
        except ValueError:
            proxy_artist = ""

        try:
            frame_type = proxy_file[proxy_file.index("(")+1:proxy_file.index(")")]
            proxy_file = proxy_file.replace(f" ({frame_type})", "")
        except ValueError:
            frame_type = ""

        name = proxy_file  # If original file name had extras, try-except blocks would remove them

        # Look up art
        try:
            versions = art[name]
        except KeyError:
            print(f"Art not found for: {name}")
            continue
        
        # Process one to many tuples of (artist, set_code) associated with card name
        if len(versions) == 1:
            artist, set_code = versions[0]
            new_file_name = f"{name} ({artist}, {set_code}){' [' + frame_type + ']' if frame_type else ''}.{file_extension}"
            os.rename(f"{typed_proxy_folder}/{original_proxy_file}", f"{destination_folder}/{new_file_name}")
        else:
            print(f"Multiple versions found: {versions}")
            continue

# Debug Commands

## Save card info to `card.json`

In [ ]:
from scripts import get_card_info

args = "\"Angelic Accord\" \"M14\""
get_card_info.main(args)

## Scryfall

### Get multiple cards

In [ ]:
from lib.helpers import scryfall_helpers

queries = [
    "Crucible of Worlds",
]

for q in queries:
    cards = scryfall_helpers.get_matched_cards(q)
    [print(f"\t{repr(c)}") for c in cards]

### Get single card

In [ ]:
from lib.helpers import scryfall_helpers

queries = [
    ("Crucible of Worlds", "5DN")
]

for name, set_code in queries:
    card = scryfall_helpers.get_named_card(name, set_code)
    print(repr(card))

## MTGPICS.com

### Find `gamerid`

In [ ]:
from lib.helpers import mtgpics_helpers

queries = [
    "Crucible of Worlds",
]

for q in queries:
    card_name, gamerid = mtgpics_helpers.get_gamerid(q)
    print(f"Found id [{gamerid}] for [{card_name}]")

### Find image versions by `gamerid`

In [ ]:
from lib.helpers import mtgpics_helpers

to_find = "Crucible of Worlds"
card_name, gamerid = to_find, mtgpics_helpers.get_gamerid(to_find)
versions = mtgpics_helpers.find_all_art_versions(card_name, gamerid)
versions

## Misc

In [1]:
import requests_cache; requests_cache.install_cache()

In [ ]:
test = "hi"
test2 = "sup"
f"{test}/{f'{test2}'}"